## Modules

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from modules import *

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\QYH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\QYH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\QYH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\QYH\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\QYH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\QYH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\QYH\AppD

## Load data

In [2]:
X,y=load_data('./data/labeledTrainData.tsv',colname=['review','sentiment'])

************** Loading Data ************


Summary of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB
No of Rows: 25000
No of Columns: 3

Data View: Last 3 Instances

              id  sentiment                                             review
24997  "10905_3"          0  "Guy is a loser. Can't get girls, needs to bui...
24998  "10194_3"          0  "This 30 minute documentary Buñuel made in the...
24999   "8478_8"          1  "I saw this movie as a child and it broke my h...

Class Counts(label, row): Total
1    12500
0    12500
Name: sentiment, dtype: int64

Data View: First 5 Instances

         id  sentiment                                             review
0  "5814_8"   

In [3]:
X_train, X_test, y_train,  y_test=split_data(X,y)


************** Spliting Data **************


************** Data After Splitting **************

Train Data: (20000, 1)
Test Data: (5000, 1)

Class Counts(label, row): Train
1    10018
0     9982
Name: sentiment, dtype: int64

Class Counts(label, row): Test
0    2518
1    2482
Name: sentiment, dtype: int64

First 5 Instance: Train
                                                  review
6167   "Some unsuspecting films carry a message that ...
3101   "Even the first 10 minutes of this movie were ...
17307  "To me this was more a wake up call, and reali...
3950   "Shower keeps within itself in so many ways. A...
893    "Brian Yuzna is often frowned upon as a direct...

First 5 Instance: Test
                                                  review
7799   "\"Girlfight\" is much more of a coming-of-age...
4427   "This movie will go down down in history as on...
14941  "I have to agree with Cal-37 it's a great movi...
11644  "Most of the Atomic Age monster movies I saw o...
15548  "I saw 

In [4]:
X_train=X_train.iloc[:, -1].apply(preprocess_data)

# word list --> str
X_train = X_train.apply(lambda x: " ".join(x))

In [5]:
X_test=X_test.iloc[:, -1].apply(preprocess_data)

# word list --> str
X_test = X_test.apply(lambda x: " ".join(x))

## Get TF-IDF vector

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
tfidf = TFIDF(min_df=2, max_features=None, strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

data_all = X_train.tolist()+X_test.tolist()
len_train = len(X_train)
tfidf.fit(data_all)
data_all = tfidf.transform(data_all)
# Revert to training set and test set parts
X_train = data_all[:len_train]
X_test = data_all[len_train:]
print('TF-IDF over')

TF-IDF over


##  Naive Bayes

In [24]:
### Naive Bayesian training ###
from sklearn.naive_bayes import MultinomialNB as MNB
model_NB = MNB()
model_NB.fit(X_train, y_train)

MultinomialNB()

In [25]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# fit
y_predict = model_NB.predict(X_test)
y_test_np = y_test.values

# evaluate
accuracy = accuracy_score(y_test_np, y_predict)
print(f"accuracy: {accuracy}")

recall = recall_score(y_test_np, y_predict)
print(f"recall: {recall}")

precision = precision_score(y_test_np, y_predict)
print(f"precision: {precision}")

f1 = f1_score(y_test_np, y_predict)
print(f"f1: {f1}")

accuracy: 0.885
recall: 0.8944399677679291
precision: 0.8764311093564943
f1: 0.8853439680957129


In [26]:
# To get a more accurate performance evaluation,
# use cross-validation to fit and evaluate evaluate the model
from sklearn.model_selection import cross_val_score
model_NB = MNB()
scores = cross_val_score(model_NB, data_all, y_train.tolist()+y_test.tolist(), cv=10)
print("10 fold cross validation score of Bayes classifier:", np.mean(scores))

10 fold cross validation score of Bayes classifier: 0.88504


## Logistic regression

In [27]:
from sklearn.linear_model import LogisticRegression

model_LR = LogisticRegression()
model_LR.fit(X_train, y_train)

LogisticRegression()

In [28]:
# fit
y_predict = model_LR.predict(X_test)
y_test_np = y_test.values

# evaluate
accuracy = accuracy_score(y_test_np, y_predict)
print(f"accuracy: {accuracy}")

recall = recall_score(y_test_np, y_predict)
print(f"recall: {recall}")

precision = precision_score(y_test_np, y_predict)
print(f"precision: {precision}")

f1 = f1_score(y_test_np, y_predict)
print(f"f1: {f1}")

accuracy: 0.885
recall: 0.9016921837228042
precision: 0.8711560918645387
f1: 0.8861611562066918


In [29]:
# To get a more accurate performance evaluation,
# use cross-validation to fit and evaluate evaluate the model
from sklearn.model_selection import cross_val_score
cross_val_model_LR = LogisticRegression()
scores = cross_val_score(cross_val_model_LR, data_all, y_train.tolist()+y_test.tolist(), cv=10)
print("10 fold cross validation score of LogisticRegression:", np.mean(scores))

10 fold cross validation score of LogisticRegression: 0.8877200000000001


In [31]:
#test
test_data=pd.read_csv("data/testData.tsv",sep = "\t")
X_test=test_data['review'].apply(preprocess_data)

# word list --> str
X_test = X_test.apply(lambda x: " ".join(x))

X_test = tfidf.transform(X_test)
test_predicted = np.array(model_LR.predict(X_test))
lr_output = pd.DataFrame(data=test_predicted, columns=['sentiment'])
lr_output['id'] = test_data['id']
lr_output = lr_output[['id', 'sentiment']]
lr_output.to_csv('tf-idf.csv', index=False)